In [2]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import statistics as st
from scipy import stats
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

#import sys
#!{sys.executable} -m pip install keras-rectified-adam

# Import custom functions
from preprocessing import *
from model import *
from evaluation import *

/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/Cellar/jupy

SyntaxError: non-default argument follows default argument (model.py, line 22)

In [ ]:
# List of training X_files
X_files = ['TrainingData/subject_001_02__x.csv', 'TrainingData/subject_001_03__x.csv', 
           'TrainingData/subject_001_04__x.csv', 'TrainingData/subject_001_05__x.csv', 
           'TrainingData/subject_001_06__x.csv', 'TrainingData/subject_001_07__x.csv', 
           'TrainingData/subject_002_01__x.csv', 'TrainingData/subject_002_03__x.csv', 
           'TrainingData/subject_002_04__x.csv', 'TrainingData/subject_002_05__x.csv', 
           'TrainingData/subject_003_01__x.csv', 'TrainingData/subject_005_02__x.csv',
           'TrainingData/subject_005_03__x.csv', 'TrainingData/subject_006_01__x.csv', 
           'TrainingData/subject_006_03__x.csv', 'TrainingData/subject_007_01__x.csv', 
           'TrainingData/subject_007_02__x.csv', 'TrainingData/subject_007_03__x.csv',
           'TrainingData/subject_001_01__x.csv', 'TrainingData/subject_003_02__x.csv', 
           'TrainingData/subject_003_03__x.csv', 'TrainingData/subject_005_01__x.csv',
           'TrainingData/subject_004_01__x.csv', 'TrainingData/subject_004_02__x.csv',
           'TrainingData/subject_008_01__x.csv', 'TrainingData/subject_007_04__x.csv',
           'TrainingData/subject_002_02__x.csv', 'TrainingData/subject_006_02__x.csv',
           'TrainingData/subject_001_08__x.csv']

# List of training X_t_files
X_t_files = ['TrainingData/subject_001_02__x_time.csv', 'TrainingData/subject_001_03__x_time.csv', 
             'TrainingData/subject_001_04__x_time.csv', 'TrainingData/subject_001_05__x_time.csv', 
             'TrainingData/subject_001_06__x_time.csv', 'TrainingData/subject_001_07__x_time.csv', 
             'TrainingData/subject_002_01__x_time.csv', 'TrainingData/subject_002_03__x_time.csv', 
             'TrainingData/subject_002_04__x_time.csv', 'TrainingData/subject_002_05__x_time.csv', 
             'TrainingData/subject_003_01__x_time.csv', 'TrainingData/subject_005_02__x_time.csv', 
             'TrainingData/subject_005_03__x_time.csv', 'TrainingData/subject_006_01__x_time.csv',  
             'TrainingData/subject_006_03__x_time.csv', 'TrainingData/subject_007_01__x_time.csv', 
             'TrainingData/subject_007_02__x_time.csv', 'TrainingData/subject_007_03__x_time.csv',
             'TrainingData/subject_001_01__x_time.csv', 'TrainingData/subject_005_01__x_time.csv',
             'TrainingData/subject_003_02__x_time.csv', 'TrainingData/subject_003_03__x_time.csv',
             'TrainingData/subject_004_01__x_time.csv', 'TrainingData/subject_004_02__x_time.csv',
             'TrainingData/subject_008_01__x_time.csv', 'TrainingData/subject_007_04__x_time.csv',
             'TrainingData/subject_002_02__x_time.csv', 'TrainingData/subject_006_02__x_time.csv',
             'TrainingData/subject_001_08__x_time.csv']

# List of training y_files
y_files = ['TrainingData/subject_001_02__y.csv', 'TrainingData/subject_001_03__y.csv',
           'TrainingData/subject_001_04__y.csv', 'TrainingData/subject_001_05__y.csv',
           'TrainingData/subject_001_06__y.csv', 'TrainingData/subject_001_07__y.csv', 
           'TrainingData/subject_002_01__y.csv', 'TrainingData/subject_002_03__y.csv', 
           'TrainingData/subject_002_04__y.csv', 'TrainingData/subject_002_05__y.csv', 
           'TrainingData/subject_003_01__y.csv', 'TrainingData/subject_005_02__y.csv', 
           'TrainingData/subject_005_03__y.csv', 'TrainingData/subject_006_01__y.csv',  
           'TrainingData/subject_006_03__y.csv', 'TrainingData/subject_007_01__y.csv', 
           'TrainingData/subject_007_02__y.csv', 'TrainingData/subject_007_03__y.csv',
           'TrainingData/subject_001_01__y.csv', 'TrainingData/subject_005_01__y.csv',
           'TrainingData/subject_003_02__y.csv', 'TrainingData/subject_003_03__y.csv',
           'TrainingData/subject_004_01__y.csv', 'TrainingData/subject_004_02__y.csv',
           'TrainingData/subject_008_01__y.csv', 'TrainingData/subject_007_04__y.csv',
           'TrainingData/subject_002_02__y.csv', 'TrainingData/subject_006_02__y.csv',
           'TrainingData/subject_001_08__y.csv']

y_t_files = ['TrainingData/subject_001_02__y_time.csv', 'TrainingData/subject_001_03__y_time.csv', 
             'TrainingData/subject_001_04__y_time.csv', 'TrainingData/subject_001_05__y_time.csv', 
             'TrainingData/subject_001_06__y_time.csv', 'TrainingData/subject_001_07__y_time.csv', 
             'TrainingData/subject_002_01__y_time.csv', 'TrainingData/subject_002_03__y_time.csv', 
             'TrainingData/subject_002_04__y_time.csv', 'TrainingData/subject_002_05__y_time.csv', 
             'TrainingData/subject_003_01__y_time.csv', 'TrainingData/subject_005_02__y_time.csv', 
             'TrainingData/subject_005_03__y_time.csv', 'TrainingData/subject_006_01__y_time.csv',  
             'TrainingData/subject_006_03__y_time.csv', 'TrainingData/subject_007_01__y_time.csv', 
             'TrainingData/subject_007_02__y_time.csv', 'TrainingData/subject_007_03__y_time.csv',
             'TrainingData/subject_001_01__y_time.csv', 'TrainingData/subject_005_01__y_time.csv',
             'TrainingData/subject_003_02__y_time.csv', 'TrainingData/subject_003_03__y_time.csv',
             'TrainingData/subject_004_01__y_time.csv', 'TrainingData/subject_004_02__y_time.csv',
             'TrainingData/subject_008_01__y_time.csv', 'TrainingData/subject_007_04__y_time.csv',
             'TrainingData/subject_002_02__y_time.csv', 'TrainingData/subject_006_02__y_time.csv',
             'TrainingData/subject_001_08__y_time.csv']

training_X, training_y = generate_data(X_files, X_t_files, y_files, y_t_files)
print(training_X.shape, training_y.shape)

In [ ]:
# Save the training data to .npy files so we do not have to generate them repeatedly
np.save('training_X.npy', training_X)
np.save('training_y.npy', training_y)

In [3]:
# Loading the data back
training_X = np.load('training_X.npy')
training_y = np.load('training_y.npy')
print(training_X.shape, training_y.shape)

(334543, 30, 6) (334543, 1)


In [5]:
# Get the class weights
label_weights = get_label_weights(training_X, training_y)

# Perform one-hot encoding
training_y_encoded = one_hot_encoding(training_y)

In [6]:
# Define the k-fold cross validator
k_fold = KFold(n_splits = 5, shuffle = True)
cv_scores_accuracy = []
cv_scores_loss = []
# K-fold cross validation  model evaluation
fold_no = 1
for train, test in k_fold.split(training_X, training_y_encoded):
    # From previous runs, we have found that dropout_rate = 0.1, l1 = 2**-6, and l2 = 2**-8 gave optimal results.
    model = define_LSTM_model(training_X, training_y_encoded, dropout_rate = 0.1, l1_value = 2**-6, l2_value = 2**-8)
    print('--------------------------------------------------------------------------------------------------------')
    print(f'Training for fold {fold_no}...')
    # Fit the model
    history = model.fit(training_X[train], training_y_encoded[train], epochs = 25, batch_size = 64, 
                        class_weight = label_weights, verbose = 0)
    # Evaluate the model, generate metrics
    scores = model.evaluate(training_X[test], training_y_encoded[test], batch_size = 64, verbose = 0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {score[0]}; {model.metrics_names[1]} of {score[1]*100}%; 
          {model.metrics_names[2]} of {score[2]}; 
          {model.metrics_names[3]} of {score[3]}; 
          {model.metrics_names[4]} of {score[4]}')
    cv_scores_accuracy.append(scores[1] * 100)
    cv_scores_loss.append(scores[0])
    # Increment the fold number
    fold_no += 1
  
"""
def evaluate_model(training_X, training_y_encoded, val_X, val_y_encoded, dropout_rate):
    verbose, epochs, batch_size = 1, 50, 64
    model = define_LSTM_model(dropout_rate)
    model.summary()
    # Fit network
    history = model.fit(training_X, training_y_encoded, epochs = epochs, batch_size = batch_size, 
              validation_data = (val_X, val_y_encoded), class_weight = label_weights, verbose = verbose)
    # Evaluate model
    loss, accuracy, f1, precision, recall = model.evaluate(val_X, val_y_encoded, 
                                                           batch_size = batch_size, verbose = verbose)
    return model, history, accuracy, f1, precision, recall
"""

SyntaxError: EOL while scanning string literal (<ipython-input-6-8833f890f07c>, line 17)

In [ ]:
# Optimal parameters: dropout_rate = 0.5, l1 = 2e-6, l2 = 2e-6
best_model, best_history, best_accuracy, best_f1, best_precision, best_recall = evaluate_model(training_X, 
                                                                                   training_y_encoded, 
                                                                                   val_X, val_y_encoded, 
                                                                                   0.1)

In [ ]:
plot_history(best_history)

In [ ]:
curr_acc = st.mean(best_history.history['val_acc'][45:50])
print(curr_acc)

In [ ]:
# Get prediction report
y_pred = best_model.predict(val_X, batch_size = 64, verbose = 1)
y_pred_bool = np.argmax(y_pred, axis = 1)
print(classification_report(val_y, y_pred_bool))

In [ ]:
validation = pd.DataFrame({'Y':val_y.flatten(), 'Prediction':y_pred_bool})
validation = pd.DataFrame(list(zip(val_y.flatten(), y_pred_bool)), columns = ['Y', 'Prediction'])
validation.to_csv('validation_prediction.csv')

In [ ]:
# Get predictions using test data
test_files = ['TestData/subject_009_01__x.csv', 'TestData/subject_010_01__x.csv', 
             'TestData/subject_011_01__x.csv', 'TestData/subject_012_01__x.csv']

y_files = ['TestData/subject_009_01__y_time.csv', 'TestData/subject_010_01__y_time.csv',
          'TestData/subject_011_01__y_time.csv', 'TestData/subject_012_01__y_time.csv']

prediction_files = ['subject_009_01__y_prediction.csv', 'subject_010_01__y_prediction.csv',
                   'subject_011_01__y_prediction.csv', 'subject_012_01__y_prediction.csv']

def generate_test_data(X, time_steps, step):
    X_values = []
    for i in range(0, len(X) - time_steps, step):
        value = X.iloc[i:(i + time_steps)].values
        X_values.append(value)
    return np.array(X_values)

def reduce(y):
    y_output = []
    for i in range(0, y.shape[0], 4):
        item = list(y[i:i + 4])
        y_output.append(max(item, key = item.count))
    return np.array(y_output)

for i in range(len(test_files)):
    test_input = pd.read_csv(test_files[i])
    test_input = scale_data(test_input, ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])
    y_frame = pd.read_csv(y_files[i])
    print(y_frame.shape)

    add_to_frame = y_frame.shape[0] * 4 - test_input.shape[0] + 30
    adding_dataframe = pd.DataFrame(test_input.iloc[-add_to_frame:])
    test_input = test_input.append(adding_dataframe)
    
    # Apply window technique
    X_test = generate_test_data(test_input, 30, 1)

    y_hat_encoded = best_model.predict(X_test, batch_size = 64, verbose = 1)
    
    y_hat = np.argmax(y_hat_encoded, axis = 1)
    
    y_actual = reduce(y_hat)
    #y_actual = y_actual[np.arange(y_actual.size - 1)]
    
    print(y_actual.shape)
    
    y_series = pd.Series(y_actual)
    y_series.to_csv("C2_predictions/" + prediction_files[i])